# How to use the College Scorecard API

Max Kapur | [maxkapur.com](https://www.maxkapur.com)

Prerequisites:

1. This tutorial assumes basic familiarity of Python and Jupyter notebooks.
1. Sign up for an API key from [https://api.data.gov/signup/](https://api.data.gov/signup/). You will get an email with your API key, which is a long string of letters and numbers.
2. Download the College Scorecard Data Dictionary from [https://collegescorecard.ed.gov/data/documentation/](https://collegescorecard.ed.gov/data/documentation/). This is an Excel spreadsheet that contains the names of all the columns available in the dataset.

In [ ]:
import requests
import pandas as pd

# Optional
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy.stats import ttest_rel

## HTTP `request.get()`

`request.get(url)` is a function that tells Python to download whatever is available at the URL. It gives us a bunch of information, including whether the request worked and the contents of the webpage (if the URL points to a webpage).

In [ ]:
resp = requests.get("https://www.youtube.com")
dir(resp)

Code 200 means OK.

In [ ]:
resp.status_code

Contents of the webpage:

In [ ]:
print(resp.text[:1000])

In [ ]:
resp = requests.get("https://www.example.com/pagethatdoesnotexist")
resp.status_code

## How to access College Scorecard data

Cobbled together from [the official API documentation](https://github.com/RTICWDT/open-data-maker/blob/master/API.md) and various examples from StackExchange.

We need to create a special URL that points to the College Scorecard database and includes our request&mdash;which information we want and from which years.

In [ ]:
base_url = "https://api.data.gov/ed/collegescorecard/v1/"
dataset = "schools.json?"
filter_params = "latest.student.size__range=25000.."
fields = ["id",
          "school.name",
          "2014.student.size",
          "2019.student.size",
          "location.lat",
          "location.lon",
          "latest.admissions.sat_scores.midpoint.math",
          "latest.admissions.act_scores.midpoint.math",
          "oops.variable.does.not.exist"]
options = "&per_page=100&page=0"
api_key = "&api_key=abc123yourkey"          # Your API key goes here.

# with open("api_key.txt") as f:
#     api_key = "&api_key=" + f.readlines()[0]
# api_key

request_url = base_url + dataset + filter_params + \
              "&fields=" + ",".join(fields) + options + api_key
print(request_url[:-40])

Common pitfalls:

- You can only pass one filter parameter at a time. If you want to filter on multiple categories, use the narrowest criterion in `filter_params` and then do the rest using the [selection tools built into Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html).

- Use `?` to introduce the query (I included it above in the variable `dataset`), then `&` to separate further elements.

- Use a comma to separate fields. I do this using `",".join(fields)` above.

- For variable names, refer to the Data Dictionary we downloaded earlier. Must preface with the year (or `latest`) and the "dev-category"&mdash;*except* for variables whose dev category is `school` (omit year) or `root` (omit year *and* dev category).

- For the latest data, you can type `latest` or the year. 

- If you type a variable name incorrectly, there is no obvious warning. It will simply be missing from the output. 

- Replace `abc123yourkey` with your API key.

In [ ]:
resp = requests.get(request_url)
# Raises an HTTPError unless status_code==200 (OK)
resp.raise_for_status()

Common errors when working with the College Scorecard database are:
- 404: You probably wrote the URL wrong.
- 429: You have made too many get requests. The default limit is 1000 per hour. A good way to avoid this is to always use `&per_page=100`.
- 500: An internal server error. Just wait a moment and try again.

## Wrangling the data into a legible format

Now `resp` contains the response from the [data.gov](https://www.data.gov) server. The server stores College Scorecard data in a format called JSON. It will be easier to do data analysis, however, if we can get the data into a Pandas data frame. We can then export to Excel or CSV if needed.

In [ ]:
resp.json()

The information we requested is stored in `resp.json()['results']`. Now we use the Python package Pandas to read this JSON file into a tabular format called a data frame. 

I changed the index of the data frame to the school ID from the database.

In [ ]:
college_info = pd.DataFrame(resp.json()['results']).set_index('id')
college_info

In [ ]:
college_info.to_excel("college_info.xlsx")

## Exploratory data analysis in Pandas

In [ ]:
college_info.describe()

In [ ]:
college_info.corr()

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(college_info['latest.admissions.act_scores.midpoint.math'],
            college_info['latest.admissions.sat_scores.midpoint.math'])

In [ ]:
has_size_info = college_info.dropna(subset=['2014.student.size', '2019.student.size']).index
ttest_rel(college_info['2014.student.size'].loc[has_size_info],
          college_info['2019.student.size'].loc[has_size_info])

According to this data, there was a statistically insignificant decline in the enrollment at large universities over the five-year period from 2014 to 2019. 

## Map the largest universities in the US

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([-130, -60, 21, 45])
ax.stock_img()
ax.gridlines()
ax.coastlines(resolution='50m')
ax.scatter(college_info['location.lon'],
           college_info['location.lat'],
           color="lightpink",
           s=4e-3*college_info['2019.student.size'],
           alpha=0.8,
           ec="black")